In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

class: center, middle

# DEMOPS: Roll decay
## Status 2020-02-27
---

### The work has been focused on:
--

### Building a roll decay DB
* SQL database with MDL model tests 2007-2020</li>

--

### Measure Rolldamping
* System identification of roll decay test
* Four methods have been tested

--

### Building roll damping DB
* System identification of roll decay DB


---
name: DB
## Building a roll decay DB
---
template: DB
<img src="mdl_db.png" alt="drawing" height="550"/>


In [2]:
from sqlalchemy import create_engine
import data
from mdldb.mdl_db import MDLDataBase
from mdldb.tables import Base, Model, LoadingCondition, Run, RolldecayLinear, RolldecayDirect, RolldecayNorwegian
import pandas as pd
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex, Markdown

engine = create_engine('sqlite:///' + data.mdl_db_path)
db = MDLDataBase(engine=engine)

df = pd.read_sql_table(table_name='run', con=engine, index_col='id')

---
template: DB

### MDL DB

In [3]:
s = """The database from MDL currently contains *%i* tests conducted between **%i** and **%i**.
"""  % (len(df),
df['date'].min().year,
df['date'].max().year,)
Markdown(s)

The database from MDL currently contains *21243* tests conducted between **2005** and **2017**.


--

In [4]:
exclude = ['smeg','prepare','rodergrundvinkel','rerun','unknown','none standard','teckenkoll']
emphase = ['roll decay']

test_type_s=''
for test_type in df['test_type'].unique():
    if test_type in exclude:
        continue
    
    if test_type in emphase:
        output = '**%s**' % test_type
    else:
        output = test_type
    
    test_type_s+=' %s,' % output

s = """<u>Test types:</u>%s
"""  % (test_type_s)
Markdown(s)

<u>Test types:</u> seakeeping, GM, reference speed, **roll decay**, zigzag, turning circle, u turn, spiral, bollard pull, captive, crash stop, wave calibration, pitch decay, surge decay, sway decay, softmooring, williamson turn, heave decay, yaw decay,


--

In [5]:
number_of_rolldecay = (df['test_type']=='roll decay').sum()
s = """**%i** of the tests are roll decay tests """ % number_of_rolldecay
Markdown(s)

**407** of the tests are roll decay tests 

In [6]:
import sympy as sp
from rolldecayestimators.symbols import *
import rolldecayestimators.equations as equations
import rolldecayestimators.direct_estimator as direct_estimator
import rolldecayestimators.direct_linear_estimator as direct_linear_estimator
import rolldecayestimators.direct_estimator_improved as direct_estimator_improved


from latex_helpers import pylatex_extenders
import latex_helpers
import os.path
from rigidbodysimulator.substitute_dynamic_symbols import substitute_dynamic_symbols, find_name, find_derivative_name, lambdify, find_derivatives
import dill

---
name: equations
## Measure Rolldamping
---
template: equations

### General equation for roll decay motion:

In [7]:
Math(vlatex(equations.roll_equation_general))

<IPython.core.display.Math object>

--

During a roll decay test external moment is zero:

--

In [8]:
Math(vlatex(equations.roll_decay_equation_general))

<IPython.core.display.Math object>

--

### Linear model

Linearizing the stiffness (ship static stability)

--


In [9]:
latex = vlatex(equations.linear_stiffness_equation)
Math(latex)

<IPython.core.display.Math object>

--

Linear roll decay equation:

In [10]:
Math(vlatex(equations.roll_decay_linear_equation))

<IPython.core.display.Math object>

---
template: equations

Damping term $ B $ can be linear:

In [11]:
Math(vlatex(sp.Eq(B,zeta)))

<IPython.core.display.Math object>

--

or quadratic:

In [12]:
Math(vlatex(sp.Eq(B,zeta+d*sp.Abs(phi_dot))))

<IPython.core.display.Math object>

---
name: nonlinearity
## Damping nonlinearity
---
template: nonlinearity
The linear model is sometimes too simple:
<img src="nonlinearity.png" alt="drawing" width=800/>

---
template: nonlinearity

In [13]:
s="""Adding quadratic damping: $%s$""" % vlatex(sp.Eq(B,zeta+d*sp.Abs(phi_dot)))
Markdown(s)

Adding quadratic damping: $B = d \left|{\dot{\phi}}\right| + \zeta$

<img src="nonlinearity_quadratic.png" alt="drawing" width=800/>

---
template: nonlinearity
<img src="nonlinearity_sweep.png" alt="drawing" height=450 width=800/>

--

* linear model has low score when including large angles

--
* linear model damping $\zeta$ increase for large angles

--
* for quadratic model $d$ increase instead



---
name: varying-stiffness
## Varying stiffness

---
template: varying-stiffness
The linear stiffness assumption is not valid for all ships at large roll angles:

---
template: varying-stiffness

In [14]:
Math(vlatex(equations.linear_stiffness_equation))

<IPython.core.display.Math object>

<img src="varying_stiffness.png" alt="drawing" width=800/>
---
template: varying-stiffness

### Natural frequency $ \omega_0 $ for each oscillation
<img src="varying_stiffness2.png" alt="drawing" width=800/>

---
template: varying-stiffness

### Adding a quadratic term to the stiffness:

In [15]:
Math(vlatex(equations.quadratic_stiffness_equation))

<IPython.core.display.Math object>

<img src="varying_stiffness3.png" alt="drawing" width=800/>

---
name: building-roll-damping-DB
## Building roll damping DB
---

In [16]:
plots = ['zeta','d','mean_damping','omega0','score']
s = ''
for plot in plots:
    code = """
template: building-roll-damping-DB
.right[![Right-aligned image](histogram_%s.png)]
---
""" % plot
    s+=code
    
Markdown(s)




template: building-roll-damping-DB
.right[![Right-aligned image](histogram_zeta.png)]
---

template: building-roll-damping-DB
.right[![Right-aligned image](histogram_d.png)]
---

template: building-roll-damping-DB
.right[![Right-aligned image](histogram_mean_damping.png)]
---

template: building-roll-damping-DB
.right[![Right-aligned image](histogram_omega0.png)]
---

template: building-roll-damping-DB
.right[![Right-aligned image](histogram_score.png)]
---


## Next steps

--

### Regression on roll damping DB
--

### Ikeda method to predict roll damping
--

### Gather more meta data
* Bilge keels
* Inertia
* Hull form (probably takes too much time)

--

### Start writing paper




---
class: center, middle
## End